# deepSimDEF for Prediction of Protein-Protein Interactions (PPI)

## Importing the required libraries

In [1]:
import os
import sys
import random
import operator
import numpy as np
import keras.backend as K

from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics import f1_score

from deepSimDEF_data_provider_PPI import gene_pair_data_reader, input_data_maker
from deepSimDEF_network import deepSimDEF_PPI
from deepSimDEF_save import save_simDEF_model, save_simDEF_embeddings

np.random.seed(321)


Using TensorFlow backend.


## Setting variables, reading GO annotations of genes, and preparing them for networks

In [2]:
FOLD = 10
DROPOUT = 0.3
MAX_POOL = True

PRE_TRAINED = True
UPDATABLE = True

ACTIVATION_HIDDEN = 'relu'
ACTIVATION_HIGHWAY = 'sigmoid'
ACTIVATION_OUTPUT = 'sigmoid'

EMBEDDING_DIM = 100
NB_EPOCH = 250
BATCH_SIZE = 256
OPTIMIZER = 'adadelta'

IEA = True
SEQ = False

TRANSFER_LEARNING = False

SAVE_MODEL = True
SAVE_EMBEDDINGS = True

SUB_ONTOLOGY = ['BP', 'CC', 'MF']
SUB_ONTOLOGY_work = ['BP', 'CC', 'MF']

WITH_HIGH_THROUPUT = False

SBOs = {}
for sbo in SUB_ONTOLOGY_work:
    if sbo == 'BP':
        SBOs[sbo] = 'Biolobical Process (BP)'
    elif sbo == 'CC':
        SBOs[sbo] = 'Cellular Component (CC)'
    elif sbo == 'MF':
        SBOs[sbo] = 'Molecular Function (MF)'
    
WE = {}
embedding_save = {}
MAX_SEQUENCE_LENGTH = {}
MAX_SEQUENCE_LENGTH_INDEX = {}
sequences = {}
word_indeces = {}
protein_index = {}    
    
for sbo in SUB_ONTOLOGY:
    WE[sbo] = 'GO_' + sbo + '_Embeddings_100D_0_6.emb'
    embedding_save[sbo] = 'GO_' + sbo + '_Embeddings_100D_0_6_updated.emb'
    MAX_SEQUENCE_LENGTH[sbo] = 0
    MAX_SEQUENCE_LENGTH_INDEX[sbo] = []
    sequences[sbo] = []
    word_indeces[sbo] = []
    protein_index[sbo] = {}
    
    if IEA:
        file_reader = open('gene_product_GO_terms_with_IEA' + '.' + sbo)
    else:
        file_reader = open('gene_product_GO_terms_without_IEA' + '.' + sbo)
    
    index_counter = 1
    texts = []
    for line in file_reader:
        values = line.rstrip().replace(':', '').split()
        protein_index[sbo][values[0]] = index_counter
        if len(values[1:]) > MAX_SEQUENCE_LENGTH[sbo]:
            MAX_SEQUENCE_LENGTH[sbo] = len(values[1:])
            MAX_SEQUENCE_LENGTH_INDEX[sbo] = index_counter
        texts.append(' '.join(values[1:]))
        index_counter += 1
        
    tokenizer = Tokenizer(lower=False, num_words=0)
    tokenizer.fit_on_texts(texts)
    sequences[sbo] = tokenizer.texts_to_sequences(texts)

    word_indeces[sbo] = tokenizer.word_index
    
    if sbo == 'BP':
        print "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Biolobical Process (BP) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    elif sbo == 'CC':
        print "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Cellular Component (CC) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    elif sbo == 'MF':
        print "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Molecular Function (MF) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
            
    print "Found " + str(len(word_indeces[sbo])) + " unique tokens in " + sbo

    MOST_FREQUENT_LEVEL = 10
    print 'Top', MOST_FREQUENT_LEVEL, 'Most Frequent GO terms annotating sequences in', sbo + ":"
    for GO_ID, indx in sorted(word_indeces[sbo].items(), key=operator.itemgetter(1))[:MOST_FREQUENT_LEVEL]:
        print '  >>>', GO_ID, '   ' ,indx
        
    print "Number of annotated gene products by '" + sbo + "' terms: " + str(len(sequences[sbo]))
    print "Maximum annotation length of one gene product ('" + sbo + "' sub-ontology):", MAX_SEQUENCE_LENGTH[sbo]
    print "Index/line of the gene product with maximum annotations ('" + sbo + "' sub-ontology):", MAX_SEQUENCE_LENGTH_INDEX[sbo]
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n"
    
    file_reader.close()
    
    
fully_annotated_sequences = []   # we keep only those genes for which we have annatoation from all ontologies (defined in SUB_ONTOLOGY variable)
for sbo in SUB_ONTOLOGY:
    fully_annotated_sequences.append(protein_index[sbo].keys())
fully_annotated_sequences = list(set(fully_annotated_sequences[0]).intersection(*fully_annotated_sequences))
print "Number of fully annotated gene products:", len(fully_annotated_sequences)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Biolobical Process (BP) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Found 3054 unique tokens in BP
Top 10 Most Frequent GO terms annotating sequences in BP:
  >>> GO0006810     1
  >>> GO0006351     2
  >>> GO0006355     3
  >>> GO0015031     4
  >>> GO0055114     5
  >>> GO0007049     6
  >>> GO0006414     7
  >>> GO0008152     8
  >>> GO0006412     9
  >>> GO0055085     10
Number of annotated gene products by 'BP' terms: 5680
Maximum annotation length of one gene product ('BP' sub-ontology): 44
Index/line of the gene product with maximum annotations ('BP' sub-ontology): 294
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Cellular Component (CC) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Found 782 unique tokens in CC
Top 10 Most Frequent GO terms annotating sequences in CC:
  >>> GO0005737     1
  >>> GO0005634     2
  >>> GO0016020     3
  >>> GO0016021     4
  >>> GO0005739     5
  >>> GO0005829     6
  >>> GO0005783 

## Reading the gene-pair PPIs: manually curated PPIs and high-throughput PPIs (optional)

In [3]:

input_data_dir = '3.PPI_data/IEA+/PPI_FULL_physical_interactions_manually_curated'
annotation_G1_dic_MC, annotation_G2_dic_MC, interaction_pr_list_MC = gene_pair_data_reader(data_dir=input_data_dir, 
                                                                                           SUB_ONTOLOGY_work=SUB_ONTOLOGY_work, 
                                                                                           fully_annotated_sequences=fully_annotated_sequences, 
                                                                                           sequences=sequences, 
                                                                                           protein_index=protein_index,
                                                                                           MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH)


VALIDATION_SPLIT = 1.0/FOLD
indices = np.arange(annotation_G1_dic_MC[sbo].shape[0])
np.random.shuffle(indices)
test_size = int(VALIDATION_SPLIT * annotation_G1_dic_MC[sbo].shape[0])


annotation_G1_dic_HT = []
annotation_G2_dic_HT = []
interaction_pr_list_HT = []

if WITH_HIGH_THROUPUT:
    input_data_dir = '3.PPI_data/IEA+/PPI_FULL_physical_interactions_high_throughput'
    annotation_G1_dic_HT, annotation_G2_dic_HT, interaction_pr_list_HT = gene_pair_data_reader(data_dir=input_data_dir, 
                                                                                               SUB_ONTOLOGY_work=SUB_ONTOLOGY_work, 
                                                                                               fully_annotated_sequences=fully_annotated_sequences, 
                                                                                               sequences=sequences, 
                                                                                               protein_index=protein_index,
                                                                                               MAX_SEQUENCE_LENGTH= MAX_SEQUENCE_LENGTH)


Shape of data tensor 1 (BP): (32956, 44)
Shape of data tensor 2 (BP): (32956, 44)
Shape of similarity tensor (BP): (32956,) 

Shape of data tensor 1 (CC): (32956, 17)
Shape of data tensor 2 (CC): (32956, 17)
Shape of similarity tensor (CC): (32956,) 

Shape of data tensor 1 (MF): (32956, 33)
Shape of data tensor 2 (MF): (32956, 33)
Shape of similarity tensor (MF): (32956,) 

Number of positive classes/interactions: 16478


## printing some information about the setting of the network and the experiment

In [4]:
for sbo in SUB_ONTOLOGY_work:
    print "@@@ " + SBOs[sbo] + " @@@"

if IEA:
    print "^^^ With IEA ^^^"
else:
    print "^^^ Without IEA ^^^"

print "%%% Optimizer:", OPTIMIZER, "%%%"

if PRE_TRAINED:
    if UPDATABLE:
        print "+++ Pre-trained (updatable) +++"
    else:
        print "+++ Pre-trained (not updatable) +++"
else:
    print "+++ NOT Pre-trained +++"

@@@ Biolobical Process (BP) @@@
@@@ Cellular Component (CC) @@@
@@@ Molecular Function (MF) @@@
^^^ With IEA ^^^
%%% Optimizer: adadelta %%%
+++ Pre-trained (updatable) +++


## Making a 10-fold cross-validation experiment

In [5]:
models = []
embedding_layers = []
bests = []
thresholds = []
B = []

for m in range(0, FOLD):
    network = deepSimDEF_PPI(EMBEDDING_DIM, 
                             model_ind=m, 
                             MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH, 
                             WORD_EMBEDDINGS=WE,
                             SUB_ONTOLOGY_work=SUB_ONTOLOGY_work,
                             word_indeces=word_indeces, 
                             ACTIVATION_HIDDEN=ACTIVATION_HIDDEN, 
                             ACTIVATION_HIGHWAY=ACTIVATION_HIGHWAY, 
                             ACTIVATION_OUTPUT=ACTIVATION_OUTPUT, 
                             DROPOUT=DROPOUT, 
                             OPTIMIZER=OPTIMIZER)
    models.append(network[0])
    embedding_layers.append(network[1])
    bests.append(0)
    thresholds.append(0)
    B.append({})

Loaded 29375 word vectors for BP (Model 1)
Loaded 4046 word vectors for CC (Model 1)
Loaded 10541 word vectors for MF (Model 1)


/root/anaconda2/lib/python2.7/site-packages/keras/legacy/layers.py:652: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


Model for Fold Number 1 Instantiated!!

Loaded 29375 word vectors for BP (Model 2)
Loaded 4046 word vectors for CC (Model 2)
Loaded 10541 word vectors for MF (Model 2)
Model for Fold Number 2 Instantiated!!

Loaded 29375 word vectors for BP (Model 3)
Loaded 4046 word vectors for CC (Model 3)
Loaded 10541 word vectors for MF (Model 3)
Model for Fold Number 3 Instantiated!!

Loaded 29375 word vectors for BP (Model 4)
Loaded 4046 word vectors for CC (Model 4)
Loaded 10541 word vectors for MF (Model 4)
Model for Fold Number 4 Instantiated!!

Loaded 29375 word vectors for BP (Model 5)
Loaded 4046 word vectors for CC (Model 5)
Loaded 10541 word vectors for MF (Model 5)
Model for Fold Number 5 Instantiated!!

Loaded 29375 word vectors for BP (Model 6)
Loaded 4046 word vectors for CC (Model 6)
Loaded 10541 word vectors for MF (Model 6)
Model for Fold Number 6 Instantiated!!

Loaded 29375 word vectors for BP (Model 7)
Loaded 4046 word vectors for CC (Model 7)
Loaded 10541 word vectors for MF (M

## Training the deepSimDEF netwrok for PPI task

In [6]:
RES = {}
best_total_f1 = 0
best_threshold = 0

early_stopping = EarlyStopping(monitor='val_loss', patience = 3)
cor = {}

best_epoch = 0

def pred(A, treshold = 0.5):
    B = []
    for n in A:
        if treshold < n:
            B.append(1)
        else:
            B.append(0)
    return B

def run_my_model(model_index, seq):
    X_train, y_train, X_test, y_test = input_data_maker(model_id=model_index, 
                                                        test_size=test_size, 
                                                        indices=indices, 
                                                        annotation_G1_dic_MC=annotation_G1_dic_MC, 
                                                        annotation_G2_dic_MC=annotation_G2_dic_MC, 
                                                        interaction_pr_list_MC=interaction_pr_list_MC, 
                                                        annotation_G1_dic_HT=annotation_G1_dic_HT,
                                                        annotation_G2_dic_HT=annotation_G2_dic_HT,
                                                        interaction_pr_list_HT=interaction_pr_list_HT,
                                                        SUB_ONTOLOGY_work=SUB_ONTOLOGY_work,
                                                        WITH_HIGH_THROUPUT=False)
    model = models[model_index]
    history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=1, validation_data=(X_test,y_test))
    p =  model.predict(X_test)
    for i in seq:
        predictions = np.asarray(pred(p, i))
        B[model_index][i] = np.round(f1_score(y_test, predictions, average='binary'), 5)
    pr = max(B[model_index].iteritems(), key=operator.itemgetter(1))[1]
    thresholds[model_index] = max(B[model_index].iteritems(), key=operator.itemgetter(1))[0]
    st = ''
    b = bests[model_index]
    if bests[model_index] < pr: 
        bests[model_index] = pr
        treshold = thresholds[model_index]
        st = "+ " + str(bests[model_index])
    else:
        st = "- " + str(bests[model_index])
    print ">>> F1-score (" + str(model_index + 1) + "):", pr, "Best (" + str(model_index + 1) + "):", st, "(" + str(thresholds[model_index]) + " : " + str(np.round(pr - b, 5)) + ")" + "\n"

def get_results(epoch_no):
    for i in seq:
        RES[i] = 0
        for j in range(FOLD):
            RES[i] += B[j][i]/FOLD
    res = max(RES.iteritems(), key=operator.itemgetter(1))[1]
    threshold_res = max(RES.iteritems(), key=operator.itemgetter(1))[0]
    cor[epoch_no + 1] = res
    total_max = 0
    for i, j in sorted(cor.items(), key=operator.itemgetter(1)):
        if total_max < j:
            total_max = j
            best_epoch = i
            threshold_best = threshold_res
    
    print "F1-score for this epoch:", res, "(", threshold_res, ")-- Best F1-score::==>", str(total_max), "(", threshold_best, ")  (for epoch #", str(best_epoch), "of", str(NB_EPOCH), "epochs)" + "\n"

def get_final_result():
    final_max = 0
    best_epoch = 0
    for i, j in sorted(cor.items(), key=operator.itemgetter(1)):
        if final_max < j:
            final_max = j
            best_epoch = i
        
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~ FINAL RESULT ~~~~~~~~~~~~~~~~~~~~~~~~~~~~" + "\n" 
    print "For embedding size '" + str(EMBEDDING_DIM) + "' best number of epochs is '" + str(i) + "' with F1-score of: " + str(final_max) +"\n"
    
for e in range(NB_EPOCH):
     
    print "~~~~~~~~~ " + '/'.join(SUB_ONTOLOGY_work) +" ~~~~~~~~~~~~~~ EPOCH " + str(e + 1) + "/" + str(NB_EPOCH) + " (Embedding dimention: " + str(EMBEDDING_DIM) + ") ~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n" 
    seq = [0.5]
    if SEQ:
        seq = np.arange(0.11, 0.9, 0.01)
        
    for index in range(0, len(models)):
        run_my_model(index, seq)
    
    get_results(e)

get_final_result()

~~~~~~~~~ BP/CC/MF ~~~~~~~~~~~~~~ EPOCH 1/250 (Embedding dimention: 100) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5417 - fmeasure: 0.7385 - val_loss: 0.4241 - val_fmeasure: 0.8119
>>> F1-score (1): 0.81198 Best (1): + 0.81198 (0.5 : 0.81198)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5505 - fmeasure: 0.7338 - val_loss: 0.4140 - val_fmeasure: 0.8279
>>> F1-score (2): 0.82917 Best (2): + 0.82917 (0.5 : 0.82917)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5636 - fmeasure: 0.7243 - val_loss: 0.4454 - val_fmeasure: 0.8020
>>> F1-score (3): 0.80284 Best (3): + 0.80284 (0.5 : 0.80284)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5499 - fmeasure: 0.7296 - val_loss

59322/59322 [==============================] - 3s - loss: 0.3586 - fmeasure: 0.8490 - val_loss: 0.3430 - val_fmeasure: 0.8579
>>> F1-score (1): 0.85829 Best (1): + 0.85829 (0.5 : 0.00591)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.3585 - fmeasure: 0.8485 - val_loss: 0.3265 - val_fmeasure: 0.8570
>>> F1-score (2): 0.85819 Best (2): + 0.85819 (0.5 : 0.00113)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.3577 - fmeasure: 0.8481 - val_loss: 0.3503 - val_fmeasure: 0.8630
>>> F1-score (3): 0.86368 Best (3): + 0.86368 (0.5 : 0.00983)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.3613 - fmeasure: 0.8466 - val_loss: 0.3137 - val_fmeasure: 0.8757
>>> F1-score (4): 0.87573 Best (4): + 0.87573 (0.5 : 0.01519)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59

59322/59322 [==============================] - 3s - loss: 0.2993 - fmeasure: 0.8780 - val_loss: 0.2860 - val_fmeasure: 0.8783
>>> F1-score (1): 0.87892 Best (1): - 0.88201 (0.5 : -0.00309)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2971 - fmeasure: 0.8785 - val_loss: 0.2917 - val_fmeasure: 0.8785
>>> F1-score (2): 0.87953 Best (2): + 0.87953 (0.5 : 0.00325)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.2981 - fmeasure: 0.8784 - val_loss: 0.3166 - val_fmeasure: 0.8780
>>> F1-score (3): 0.87846 Best (3): + 0.87846 (0.5 : 0.00465)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.3012 - fmeasure: 0.8763 - val_loss: 0.2666 - val_fmeasure: 0.8989
>>> F1-score (4): 0.89839 Best (4): + 0.89839 (0.5 : 0.0153)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59

59322/59322 [==============================] - 3s - loss: 0.2610 - fmeasure: 0.8937 - val_loss: 0.2666 - val_fmeasure: 0.8885
>>> F1-score (1): 0.88868 Best (1): - 0.89238 (0.5 : -0.0037)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2608 - fmeasure: 0.8946 - val_loss: 0.2721 - val_fmeasure: 0.8882
>>> F1-score (2): 0.88896 Best (2): + 0.88896 (0.5 : 0.00014)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2594 - fmeasure: 0.8950 - val_loss: 0.3007 - val_fmeasure: 0.8836
>>> F1-score (3): 0.8843 Best (3): + 0.8843 (0.5 : 0.00329)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2554 - fmeasure: 0.8981 - val_loss: 0.2465 - val_fmeasure: 0.9045
>>> F1-score (4): 0.9044 Best (4): + 0.9044 (0.5 : 0.00458)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 

59322/59322 [==============================] - 3s - loss: 0.2319 - fmeasure: 0.9091 - val_loss: 0.2546 - val_fmeasure: 0.8957
>>> F1-score (1): 0.89583 Best (1): - 0.89685 (0.5 : -0.00102)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2316 - fmeasure: 0.9081 - val_loss: 0.2609 - val_fmeasure: 0.8985
>>> F1-score (2): 0.89904 Best (2): - 0.89952 (0.5 : -0.00048)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2319 - fmeasure: 0.9080 - val_loss: 0.2752 - val_fmeasure: 0.8952
>>> F1-score (3): 0.89554 Best (3): + 0.89554 (0.5 : 0.00672)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2300 - fmeasure: 0.9099 - val_loss: 0.2354 - val_fmeasure: 0.9082
>>> F1-score (4): 0.90775 Best (4): - 0.91006 (0.5 : -0.00231)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322

...

59322/59322 [==============================] - 4s - loss: 0.0170 - fmeasure: 0.9938 - val_loss: 0.4567 - val_fmeasure: 0.9250
>>> F1-score (1): 0.92552 Best (1): - 0.92906 (0.5 : -0.00354)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.0171 - fmeasure: 0.9938 - val_loss: 0.5086 - val_fmeasure: 0.9182
>>> F1-score (2): 0.91865 Best (2): - 0.92271 (0.5 : -0.00406)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0172 - fmeasure: 0.9938 - val_loss: 0.5661 - val_fmeasure: 0.9133
>>> F1-score (3): 0.91398 Best (3): - 0.91951 (0.5 : -0.00553)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0169 - fmeasure: 0.9942 - val_loss: 0.4837 - val_fmeasure: 0.9235
>>> F1-score (4): 0.92353 Best (4): - 0.92927 (0.5 : -0.00574)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

59322/59322 [==============================] - 3s - loss: 0.0186 - fmeasure: 0.9936 - val_loss: 0.4522 - val_fmeasure: 0.9262
>>> F1-score (1): 0.92668 Best (1): - 0.92906 (0.5 : -0.00238)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0180 - fmeasure: 0.9937 - val_loss: 0.5238 - val_fmeasure: 0.9183
>>> F1-score (2): 0.91877 Best (2): - 0.92271 (0.5 : -0.00394)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0171 - fmeasure: 0.9937 - val_loss: 0.5744 - val_fmeasure: 0.9141
>>> F1-score (3): 0.91457 Best (3): - 0.91951 (0.5 : -0.00494)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0182 - fmeasure: 0.9934 - val_loss: 0.4727 - val_fmeasure: 0.9260
>>> F1-score (4): 0.92609 Best (4): - 0.92927 (0.5 : -0.00318)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

59322/59322 [==============================] - 3s - loss: 0.0183 - fmeasure: 0.9936 - val_loss: 0.4368 - val_fmeasure: 0.9261
>>> F1-score (1): 0.92648 Best (1): - 0.92906 (0.5 : -0.00258)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.0181 - fmeasure: 0.9933 - val_loss: 0.5191 - val_fmeasure: 0.9189
>>> F1-score (2): 0.91945 Best (2): - 0.92271 (0.5 : -0.00326)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0184 - fmeasure: 0.9936 - val_loss: 0.5772 - val_fmeasure: 0.9150
>>> F1-score (3): 0.91573 Best (3): - 0.91951 (0.5 : -0.00378)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.0184 - fmeasure: 0.9935 - val_loss: 0.4657 - val_fmeasure: 0.9268
>>> F1-score (4): 0.92679 Best (4): - 0.92927 (0.5 : -0.00248)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

59322/59322 [==============================] - 3s - loss: 0.0164 - fmeasure: 0.9943 - val_loss: 0.4485 - val_fmeasure: 0.9248
>>> F1-score (1): 0.92528 Best (1): - 0.92906 (0.5 : -0.00378)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0164 - fmeasure: 0.9939 - val_loss: 0.5342 - val_fmeasure: 0.9190
>>> F1-score (2): 0.91959 Best (2): - 0.92271 (0.5 : -0.00312)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0171 - fmeasure: 0.9937 - val_loss: 0.5604 - val_fmeasure: 0.9133
>>> F1-score (3): 0.91386 Best (3): - 0.91951 (0.5 : -0.00565)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0171 - fmeasure: 0.9939 - val_loss: 0.4785 - val_fmeasure: 0.9256
>>> F1-score (4): 0.92582 Best (4): - 0.92927 (0.5 : -0.00345)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

59322/59322 [==============================] - 3s - loss: 0.0169 - fmeasure: 0.9937 - val_loss: 0.4570 - val_fmeasure: 0.9265
>>> F1-score (1): 0.92699 Best (1): - 0.92906 (0.5 : -0.00207)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0158 - fmeasure: 0.9943 - val_loss: 0.5276 - val_fmeasure: 0.9177
>>> F1-score (2): 0.91818 Best (2): - 0.92271 (0.5 : -0.00453)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0165 - fmeasure: 0.9941 - val_loss: 0.5740 - val_fmeasure: 0.9151
>>> F1-score (3): 0.91563 Best (3): - 0.91951 (0.5 : -0.00388)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0177 - fmeasure: 0.9937 - val_loss: 0.4985 - val_fmeasure: 0.9243
>>> F1-score (4): 0.92443 Best (4): - 0.92927 (0.5 : -0.00484)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

## Saving the model and the embeddings

In [10]:
if SAVE_MODEL:
    save_simDEF_model(FOLD=FOLD, models=models)
    
if SAVE_EMBEDDINGS:
    save_simDEF_embeddings(FOLD=FOLD, 
                           embedding_layers=embedding_layers,
                           word_indeces=word_indeces, 
                           SUB_ONTOLOGY_work=SUB_ONTOLOGY_work)
    

Saving model 1 to disk ...
The Model and its Weights Are Saved!!

Saving model 2 to disk ...
The Model and its Weights Are Saved!!

Saving model 3 to disk ...
The Model and its Weights Are Saved!!

Saving model 4 to disk ...
The Model and its Weights Are Saved!!

Saving model 5 to disk ...
The Model and its Weights Are Saved!!

Saving model 6 to disk ...
The Model and its Weights Are Saved!!

Saving model 7 to disk ...
The Model and its Weights Are Saved!!

Saving model 8 to disk ...
The Model and its Weights Are Saved!!

Saving model 9 to disk ...
The Model and its Weights Are Saved!!

Saving model 10 to disk ...
The Model and its Weights Are Saved!!

The Word Embeddings Are Saved!!
